In [1]:
import numpy as np
from scipy.optimize import minimize
import time
import pandas as pd

# Objective function
def objective(x):
    x1, x2, x3 = x
    return x1**2 + x2**2 + x3

# Penalty method
def penalty(x, lambdas):
    x1, x2, x3 = x
    g1 = x1 + x2 + x3 - 4
    g2 = 2*x1 - 3*x2 - 12
    return lambdas[0] * g1**2 + lambdas[1] * g2**2

def penalty_objective(x, lambdas):
    return objective(x) + penalty(x, lambdas)

def solve_penalty(x0, lambdas):
    start_time = time.time()
    result = minimize(penalty_objective, x0, args=(lambdas,), method='SLSQP')
    end_time = time.time()
    return result.x, objective(result.x), end_time - start_time

# Gradient projection method
def grad_objective(x):
    x1, x2, x3 = x
    return np.array([2*x1, 2*x2, 1])

def constraints(x):
    x1, x2, x3 = x
    return [x1 + x2 + x3 - 4, 2*x1 - 3*x2 - 12]

def projection(x):
    cons = [{'type': 'eq', 'fun': lambda x: x[0] + x[1] + x[2] - 4},
            {'type': 'eq', 'fun': lambda x: 2*x[0] - 3*x[1] - 12}]
    result = minimize(lambda y: np.linalg.norm(y - x), x, constraints=cons)
    return result.x

def gradient_projection(x0, alpha=0.01, tol=1e-6, max_iter=1000):
    x = np.array(x0)
    start_time = time.time()
    for _ in range(max_iter):
        grad = grad_objective(x)
        x_new = x - alpha * grad
        x_proj = projection(x_new)
        if np.linalg.norm(x_proj - x) < tol:
            break
        x = x_proj
    end_time = time.time()
    return x, objective(x), end_time - start_time

# Initial guess
x0 = [0, 0, 0]

# Solve using each method
lambdas = [100, 100]

penalty_solution, penalty_objective_value, penalty_time = solve_penalty(x0, lambdas)
gradient_solution, gradient_objective_value, gradient_time = gradient_projection(x0)

# Create a comparison table
data = {
    "Method": ["Penalty Method", "Gradient Projection"],
    "Solution (x1, x2, x3)": [penalty_solution, gradient_solution],
    "Objective Value": [penalty_objective_value, gradient_objective_value],
    "Time (s)": [penalty_time, gradient_time]
}

df = pd.DataFrame(data)

df

,Method,"Solution (x1, x2, x3)",Objective Value,Time (s)
0,Penalty Method,"[2.421593684700156, -2.3824015491189847, 3.955...",15.495761,0.000000
1,Gradient Projection,"[2.4338233897015726, -2.377451073532288, 3.943...",15.519398,4.798217


In [3]:
import cvxpy as cp

In [4]:
# Определяем переменные
x1 = cp.Variable()
x2 = cp.Variable()
x3 = cp.Variable()

# Определяем целевую функцию
objective = cp.Minimize(x1**2 + x2**2 + x3)

# Определяем ограничения
constraints = [
    x1 + x2 + x3 == 4,
    2*x1 - 3*x2 == 12
]

# Определяем проблему
problem = cp.Problem(objective, constraints)

# Решаем проблему
result = problem.solve()

# Выводим результаты
optimal_solution = [x1.value, x2.value, x3.value]
objective_value = objective.value
optimal_solution, objective_value

([array(2.42307692), array(-2.38461538), array(3.96153846)], 15.51923076923077)